In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [ ]:

# Загрузка данных
X_train = np.load("X_train_smooth.npy")
X_test = np.load("X_test_smooth.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# Даунсэмплинг: каждая 64-я точка → 65536 / 64 = 1024
X_train_down = X_train[:, ::64]  # (207, 1024)
X_test_down = X_test[:, ::64]    # (53, 1024)

X_train_tensor = torch.tensor(X_train_down, dtype=torch.float32).unsqueeze(1)  # (N, 1, T)
X_test_tensor = torch.tensor(X_test_down, dtype=torch.float32).unsqueeze(1)    # (N, 1, T)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)        # (N, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)          # (N, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
class SignalCNN(nn.Module):
    def __init__(self):
        super(SignalCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=3)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)  

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.global_pool = nn.AdaptiveAvgPool1d(1)

        self.flatten = nn.Flatten() 

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool1(x)

        x = self.relu2(self.conv2(x))
        x = self.pool2(x)

        x = self.relu3(self.conv3(x))
        x = self.pool3(x)

        x = self.global_pool(x)  
        x = self.flatten(x)     
        x = self.dropout(x)
        x = self.relu4(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SignalCNN()
print(model)

SignalCNN(
  (conv1): Conv1d(1, 32, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu3): ReLU()
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_pool): AdaptiveAvgPool1d(output_size=1)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [7]:
# ОБУЧЕНИЕ
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  

epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            predicted = (outputs > 0.5).float()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    accuracy = 100 * correct / total

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{epochs}], "
              f"Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Accuracy: {accuracy:.2f}%")
# Сохраняем модель
torch.save(model.state_dict(), "signal_cnn.pth")
print("✅ Модель сохранена как 'signal_cnn.pth'")

Epoch [0/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [10/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [20/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [30/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [40/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [50/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [60/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [70/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [80/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [90/100], Train Loss: 0.4808, Val Loss: 0.0000, Accuracy: 100.00%
✅ Модель сохранена как 'signal_cnn.pth'


In [6]:
# Точность на тесте
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

test_accuracy = 100 * correct / total
print(f"\n✅ Тестовая точность: {test_accuracy:.2f}%")


✅ Тестовая точность: 98.11%
